## Lab 03: TensorFlow vs. PyTorch

• Build and train the same small neural network model using both TensorFlow and PyTorch.

• Measure and compare training time and performance.

• Convert the trained models into lightweight formats suitable for embedded deployment: Tensor-
Flow Lite and ONNX.

## Tensorflow Implementation

In [16]:
import tensorflow as tf
# Use the MNIST dataset of handwritten digits (28x28 grayscale images, 10 classes).
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

In [17]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255   # Fill in normalization factor
x_test = x_test / 255     # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [18]:
# Architecture
# • Flatten input (784 features)
# • Dense layer with 64 ReLU units
# • Output layer with 10 units (softmax for TensorFlow)

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),              # Input shape: 28x28 pixels
    tf.keras.layers.Flatten(),                          # Flatten to 784 features
    tf.keras.layers.Dense(64, activation='relu'),       # Hidden layer with 64 neurons
    tf.keras.layers.Dense(10, activation='softmax')     # Output layer: 10 classes (digits 0–9)
])


In [20]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',       # Fill name of loss function (# Because labels are one-hot encoded)
              metrics=['accuracy'])

start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end-start:.2f} seconds")       # Output training time


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8605 - loss: 0.5048
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9565 - loss: 0.1490
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9690 - loss: 0.1072
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9771 - loss: 0.0787
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9806 - loss: 0.0635
TF Training time: 29.95 seconds


 Tensorflow - Test accuracy and Inference time using TensorFlow’s model.evaluate()

In [26]:
# Measure inference time and get evaluation metrics
start_inference = time.time()

# model.evaluate runs inference on the test set and returns [loss, accuracy]
test_loss, test_accuracy = model.evaluate(x_test, y_test)

end_inference = time.time()
inference_time = end_inference - start_inference

# Print results
print(f"Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Accuracy in Percentage: {test_accuracy * 100:.2f}%")
print(f"Inference Time on Test Set: {inference_time:.2f} seconds")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9683 - loss: 0.1015
Loss: 0.08883529156446457
Test Accuracy: 0.9722999930381775
Test Accuracy in Percentage: 97.23%
Inference Time on Test Set: 1.07 seconds


Convert the trained model to TensorFlow Lite using TFLiteConverter.

In [23]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpnvx7udma'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  133417729333136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133417729333712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133417729333904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133417729335056: TensorSpec(shape=(), dtype=tf.resource, name=None)


## Pytorch Implementation

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_loader = DataLoader(datasets.MNIST(root='./data', train=True, transform=transform, download=True), batch_size=32)
test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transform, download=True), batch_size=1000)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)    # Fill correct input and output size
        self.fc2 = nn.Linear(64, 10)    # Fill correct input and output size
    def forward(self, x):
        x = F.relu(self.fc1(x))    # Fill correct layer
        return self.fc2(x)    # Fill correct layer

model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

start = time.time()

for epoch in range(5):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(pred.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}:  Accuracy = {epoch_acc} - Loss = {epoch_loss}")

end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

Epoch 1:  Accuracy = 90.68333333333334 - Loss = 0.34226688976734876
Epoch 2:  Accuracy = 95.075 - Loss = 0.1697697012681514
Epoch 3:  Accuracy = 96.36333333333333 - Loss = 0.1245271462171028
Epoch 4:  Accuracy = 97.15166666666667 - Loss = 0.097628630991932
Epoch 5:  Accuracy = 97.68666666666667 - Loss = 0.07935793280173094
PyTorch Training time: 47.98 seconds


Inference and Evaluation using PyTorch’s model.eval() + torch.no_grad().

In [11]:
#Inference and Evaluation using PyTorch’s model.eval() + torch.no_grad().
model.eval()
correct = 0

start_time = time.time()

with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()

end_time = time.time()

inference_time = end_time - start_time

print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")
print(f"Inference Time: {inference_time:.4f} seconds")

Test accuracy: 0.9661
Inference Time: 1.0223 seconds


convert the trained model into lightweight formats - using ONNX
1. Export the model to ONNX format.
2. Use dummy input with correct shape (e.g. torch.randn(1, 784)).
3. Save as model.onnx.

In [12]:
# Install ONNX
!pip install onnx

In [13]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])
print('Successfully saved the model as model.onnx')

Successfully saved the model as model.onnx


# TensorFlow custom training loop using tf.GradientTape

In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255   # Fill in normalization factor
x_test = x_test / 255   # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets
batch_size = 32         # Fill same batch size as in first TF example
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3947, Accuracy: 0.0625
Step 100, Loss: 0.5532, Accuracy: 0.7119
Step 200, Loss: 0.4767, Accuracy: 0.7920
Step 300, Loss: 0.2381, Accuracy: 0.8253
Step 400, Loss: 0.3819, Accuracy: 0.8461
Step 500, Loss: 0.5366, Accuracy: 0.8595
Step 600, Loss: 0.2839, Accuracy: 0.8697
Step 700, Loss: 0.3174, Accuracy: 0.8758
Step 800, Loss: 0.3370, Accuracy: 0.8817
Step 900, Loss: 0.3957, Accuracy: 0.8863
Step 1000, Loss: 0.2489, Accuracy: 0.8911
Step 1100, Loss: 0.1612, Accuracy: 0.8954
Step 1200, Loss: 0.2258, Accuracy: 0.8987
Step 1300, Loss: 0.0997, Accuracy: 0.9016
Step 1400, Loss: 0.1841, Accuracy: 0.9037
Step 1500, Loss: 0.0540, Accuracy: 0.9065
Step 1600, Loss: 0.4140, Accuracy: 0.9091
Step 1700, Loss: 0.2169, Accuracy: 0.9111
Step 1800, Loss: 0.1698, Accuracy: 0.9127
Training Accuracy for epoch 1: 0.9143

Epoch 2/5
Step 0, Loss: 0.0627, Accuracy: 0.9688
Step 100, Loss: 0.1259, Accuracy: 0.9446
Step 200, Loss: 0.3803, Accuracy: 0.9510
Step 300, Loss: 0.1471, Accuracy:

## Performance Optimization with Graph Execution using @tf.function

In [14]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255   # Fill in normalization factor
x_test = x_test / 255   # Fill in normalization factor
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compile the function into a graph
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.4904, Accuracy: 0.0625
Step 100, Loss: 0.6078, Accuracy: 0.7101
Step 200, Loss: 0.4359, Accuracy: 0.7886
Step 300, Loss: 0.2628, Accuracy: 0.8275
Step 400, Loss: 0.4302, Accuracy: 0.8499
Step 500, Loss: 0.3495, Accuracy: 0.8623
Step 600, Loss: 0.0857, Accuracy: 0.8720
Step 700, Loss: 0.4313, Accuracy: 0.8786
Step 800, Loss: 0.6529, Accuracy: 0.8849
Step 900, Loss: 0.0978, Accuracy: 0.8894
Step 1000, Loss: 0.2522, Accuracy: 0.8937
Step 1100, Loss: 0.1169, Accuracy: 0.8974
Step 1200, Loss: 0.1607, Accuracy: 0.8992
Step 1300, Loss: 0.2570, Accuracy: 0.9021
Step 1400, Loss: 0.3937, Accuracy: 0.9047
Step 1500, Loss: 0.3342, Accuracy: 0.9073
Step 1600, Loss: 0.0966, Accuracy: 0.9095
Step 1700, Loss: 0.1676, Accuracy: 0.9117
Step 1800, Loss: 0.1464, Accuracy: 0.9142
Training Accuracy for epoch 1: 0.9156

Epoch 2/5
Step 0, Loss: 0.2268, Accuracy: 0.9062
Step 100, Loss: 0.1442, Accuracy: 0.9533
Step 200, Loss: 0.3749, Accuracy: 0.9498
Step 300, Loss: 0.0738, Accuracy: